In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from seapopym.configuration.no_transport import ForcingParameter, ForcingUnit

from seapopym_optimization.constraint.energy_transfert_constraint import EnergyCoefficientConstraint
from seapopym_optimization.cost_function import SimpleRootMeanSquareErrorCostFunction, TimeSeriesObservation
from seapopym_optimization.functional_group import NoTransportFunctionalGroup, Parameter
from seapopym_optimization.functional_group.base_functional_group import FunctionalGroupSet
from seapopym_optimization.genetic_algorithm.simple_genetic_algorithm import (
    SimpleGeneticAlgorithm,
    SimpleGeneticAlgorithmParameters,
)
from seapopym_optimization.genetic_algorithm.simple_logbook import Logbook
from seapopym_optimization.model_generator import NoTransportModelGenerator


/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.13/site-packages/seapopym/standard/units.py:9: UserWarning: Import(s) unavailable to set up matplotlib support...skipping this portion of the setup.
  import cf_xarray.units  # noqa: F401


In [2]:
import logging

logging.basicConfig(level=logging.WARNING, format="\n%(asctime)s - %(name)s - %(levelname)s - %(message)s")
logger = logging.getLogger("seapopym_optimization")
logger.setLevel(logging.DEBUG)

In [3]:
fg1 = NoTransportFunctionalGroup(
    name="FG1",
    day_layer=0,
    night_layer=0,
    energy_transfert=Parameter(name="FG1_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

fg2 = NoTransportFunctionalGroup(
    name="FG2",
    day_layer=1,
    night_layer=1,
    energy_transfert=Parameter(name="FG2_energy_transfert", lower_bound=0, upper_bound=1),
    lambda_temperature_0=Parameter(name="lambda_temperature_0", lower_bound=0, upper_bound=100),
    gamma_lambda_temperature=Parameter(name="gamma_lambda_temperature", lower_bound=0, upper_bound=1),
    tr_0=Parameter(name="tr_0", lower_bound=0, upper_bound=10),
    gamma_tr=Parameter(name="gamma_tr", lower_bound=-1, upper_bound=0),
)

In [4]:
constraint = EnergyCoefficientConstraint(
    parameters_name=["FG1_energy_transfert", "FG2_energy_transfert"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

In [5]:
fg_set = FunctionalGroupSet(
    functional_groups=[fg1, fg2],
)

In [6]:
fg_set.unique_functional_groups_parameters_ordered().keys()

dict_keys(['FG1_energy_transfert', 'lambda_temperature_0', 'gamma_lambda_temperature', 'tr_0', 'gamma_tr', 'FG2_energy_transfert'])

In [7]:
parameters_set = [1, np.inf, 1, 1, -1, 1]
functional_group_parameters = fg_set.generate(parameters_set)
functional_group_parameters

[{'day_layer': 0,
  'night_layer': 0,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1},
 {'day_layer': 1,
  'night_layer': 1,
  'energy_transfert': 1,
  'lambda_temperature_0': inf,
  'gamma_lambda_temperature': 1,
  'tr_0': 1,
  'gamma_tr': -1}]

In [8]:
nb_days = 360
temperature = xr.DataArray(
    data=np.zeros((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "depth"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "depth": [0, 1],
    },
    name="temperature",
    attrs={
        "units": "Celsius",
        "long_name": "Sea surface temperature",
        "standard_name": "sea_surface_temperature",
    },
)
primary_production = xr.DataArray(
    data=np.ones((nb_days, 1, 1)),
    dims=["time", "latitude", "longitude"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
    },
    name="primary_production",
    attrs={
        "units": "kg/m^2/day",
        "long_name": "Primary production",
        "standard_name": "primary_production",
    },
)
observation = xr.DataArray(
    data=np.ones((nb_days, 1, 1, 2)),
    dims=["time", "latitude", "longitude", "layer"],
    coords={
        "time": pd.date_range("2023-01-01", periods=nb_days, freq="D"),
        "latitude": [0],
        "longitude": [0],
        "layer": [0, 1],
    },
    attrs={
        "units": "kg/m^2",
    },
)
observation.time.attrs = {"axis": "T"}
temperature.time.attrs = {"axis": "T"}
primary_production.time.attrs = {"axis": "T"}

observation.latitude.attrs = {"axis": "Y"}
temperature.latitude.attrs = {"axis": "Y"}
primary_production.latitude.attrs = {"axis": "Y"}

observation.longitude.attrs = {"axis": "X"}
temperature.longitude.attrs = {"axis": "X"}
primary_production.longitude.attrs = {"axis": "X"}

observation.layer.attrs = {"axis": "Z"}
temperature.depth.attrs = {"axis": "Z"}

forcing_parameter = ForcingParameter(
    temperature=ForcingUnit(forcing=temperature),
    primary_production=ForcingUnit(forcing=primary_production),
)
with xr.set_options(keep_attrs=True):
    observation_2 = observation.sel(layer=[1]) / 2

In [9]:
observation_1 = TimeSeriesObservation(
    name="Observation_all",
    observation=observation,
)
observation_2 = TimeSeriesObservation(
    name="Observation_meso",
    observation=observation_2,
)

In [10]:
model_generator = NoTransportModelGenerator(forcing_parameters=forcing_parameter)

In [11]:
model = model_generator.generate(functional_group_parameters=functional_group_parameters)

In [12]:
model.state

<xarray.Dataset> Size: 12kB
Dimensions:                     (time: 360, latitude: 1, longitude: 1,
                                 depth: 2, functional_group: 2, cohort: 1)
Coordinates:
  * latitude                    (latitude) int64 8B 0
  * longitude                   (longitude) int64 8B 0
  * depth                       (depth) int64 16B 0 1
  * time                        (time) datetime64[ns] 3kB 2023-01-01 ... 2023...
  * cohort                      (cohort) int64 8B 0
  * functional_group            (functional_group) int64 16B 0 1
Data variables: (12/18)
    temperature                 (time, latitude, longitude, depth) float64 6kB ...
    primary_production          (time, latitude, longitude) float64 3kB 1.0 ....
    name                        (functional_group) <U7 56B 'Group_0' 'Group_1'
    energy_transfert            (functional_group) int64 16B 1 1
    lambda_temperature_0        (functional_group) float64 16B inf inf
    gamma_lambda_temperature    (functional_group) int64 16B 1 1
    ...                          ...
    max_timestep                (functional_group, cohort) int64 16B 1 1
    mean_timestep               (functional_group, cohort) float64 16B 1.0 1.0
    timestep                    int64 8B 1
    angle_horizon_sun           float64 8B 0.0
    compute_initial_conditions  bool 1B False
    compute_preproduction       bool 1B False

In [13]:
cost_function = SimpleRootMeanSquareErrorCostFunction(
    model_generator=model_generator,
    observations=[observation_1, observation_2],
    functional_groups=[fg1, fg2],
    root_mse=True,
    centered_mse=False,
    normalized_mse=False,
)
partial_cost_function = cost_function.generate()
partial_cost_function(parameters_set)


(np.float64(0.0), np.float64(0.5))

In [14]:
metaparam = SimpleGeneticAlgorithmParameters(
    ETA=4, INDPB=0.2, CXPB=0.5, MUTPB=0.2, NGEN=3, POP_SIZE=10, cost_function_weight=(-1, -1)
)
genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam, cost_function=cost_function, constraint=[constraint]
)


In [15]:
genetic_algorithm.optimize()

In [16]:
genetic_algorithm.logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.097099   
                                       1                      0.616272   
                                       2                      0.148054   
                                       3                      0.668961   
                                       4                      0.412730   
                                       5                      0.406171   
                                       6                      0.280717   
                                       7                      0.245063   
                                       8                      0.510089   
                                       9                      0.987483   
1          True                        0                      0.510089   
                                       1                      0.510089   
                                       2                      0.406171   
           False                       3                      0.668961   
                                       4                      0.406171   
                                       5                      0.510089   
                                       6                      0.280717   
                                       7                      0.668961   
                                       8                      0.987483   
                                       9                      0.406171   
2          False                       0                      0.406171   
                                       1                      0.668961   
                                       2                      0.280717   
                                       3                      0.510089   
                                       4                      0.510089   
                                       5                      0.406171   
           True                        6                      0.510089   
                                       7                      0.510089   
           False                       8                      0.406171   
                                       9                      0.554510   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      5.528275   
                                       1                     70.099327   
                                       2                     16.241361   
                                       3                      5.674832   
                                       4                     78.338013   
                                       5                     48.748720   
                                       6                     95.423566   
                                       7                      0.771402   
                                       8                     43.243522   
                                       9                      7.045557   
1          True                        0                     43.243522   
                                       1                     43.243522   
                                       2                     48.748720   
           False                       3                      5.674832   
                                       4                     48.748720   
                                       5                     57.286635   
                                       6                     95.423566   
                                       7                      5.674832   
   

In [17]:
unevaluated_logbook = genetic_algorithm.logbook.loc[[0]].copy()
unevaluated_logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.097099   
                                       1                      0.616272   
                                       2                      0.148054   
                                       3                      0.668961   
                                       4                      0.412730   
                                       5                      0.406171   
                                       6                      0.280717   
                                       7                      0.245063   
                                       8                      0.510089   
                                       9                      0.987483   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      5.528275   
                                       1                     70.099327   
                                       2                     16.241361   
                                       3                      5.674832   
                                       4                     78.338013   
                                       5                     48.748720   
                                       6                     95.423566   
                                       7                      0.771402   
                                       8                     43.243522   
                                       9                      7.045557   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          False                       0                          0.221731   
                                       1                          0.646643   
                                       2                          0.596979   
                                       3                          0.358518   
                                       4                          0.224812   
                                       5                          0.259544   
                                       6                          0.065504   
                                       7                          0.272470   
                                       8                          0.778901   
                                       9                          0.538204   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          False                       0           8.255271 -0.680228   
                                       1           6.922715 -0.113978   
                                       2           9.500077 -0.823565   
                                       3           3.895901 -0.331902   
                                       4           4.280755 -0.581290   
                                       5           5.413022 -0.835979   
                                       6           5.259739 -0.493918   
                                       7           3.115367 -0.268690   
                                       8           0.805111 -0.463210   
                                       9           2.298320 -0.217734   

category                                                                \
name                               

In [ ]:
unevaluated_logbook.iloc[:5, -3:] = np.nan
unevaluated_logbook = Logbook(unevaluated_logbook)
unevaluated_logbook

category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.097099   
                                       1                      0.616272   
                                       2                      0.148054   
                                       3                      0.668961   
                                       4                      0.412730   
                                       5                      0.406171   
                                       6                      0.280717   
                                       7                      0.245063   
                                       8                      0.510089   
                                       9                      0.987483   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      5.528275   
                                       1                     70.099327   
                                       2                     16.241361   
                                       3                      5.674832   
                                       4                     78.338013   
                                       5                     48.748720   
                                       6                     95.423566   
                                       7                      0.771402   
                                       8                     43.243522   
                                       9                      7.045557   

category                                                                    \
name                                              gamma_lambda_temperature   
Generation Is_From_Previous_Generation Individual                            
0          False                       0                          0.221731   
                                       1                          0.646643   
                                       2                          0.596979   
                                       3                          0.358518   
                                       4                          0.224812   
                                       5                          0.259544   
                                       6                          0.065504   
                                       7                          0.272470   
                                       8                          0.778901   
                                       9                          0.538204   

category                                                               \
name                                                   tr_0  gamma_tr   
Generation Is_From_Previous_Generation Individual                       
0          False                       0           8.255271 -0.680228   
                                       1           6.922715 -0.113978   
                                       2           9.500077 -0.823565   
                                       3           3.895901 -0.331902   
                                       4           4.280755 -0.581290   
                                       5           5.413022 -0.835979   
                                       6           5.259739 -0.493918   
                                       7           3.115367 -0.268690   
                                       8           0.805111 -0.463210   
                                       9           2.298320 -0.217734   

category                                                                \
name                               

In [19]:
new_genetic_algorithm = SimpleGeneticAlgorithm(
    meta_parameter=metaparam,
    cost_function=cost_function,
    constraint=[constraint],
    logbook=unevaluated_logbook,
)


In [20]:
new_genetic_algorithm.optimize()


2025-06-06 12:23:56,041 - seapopym_optimization.genetic_algorithm.simple_genetic_algorithm - INFO - Logbook found. Loading last generation.

2025-06-06 12:23:56,046 - seapopym_optimization.genetic_algorithm.simple_genetic_algorithm - WARNING - Some individuals in the logbook have no fitness values. Re-evaluating the population.


In [21]:
new_genetic_algorithm.logbook


category                                                     Parametre  \
name                                              FG1_energy_transfert   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      0.097099   
                                       1                      0.616272   
                                       2                      0.148054   
                                       3                      0.668961   
                                       4                      0.412730   
           True                        5                      0.406171   
                                       6                      0.280717   
                                       7                      0.245063   
                                       8                      0.510089   
                                       9                      0.987483   
1          True                        0                      0.510089   
           False                       1                      0.245063   
                                       2                      0.201559   
           True                        3                      0.510089   
           False                       4                      0.148054   
                                       5                      0.148054   
                                       6                      0.280717   
           True                        7                      0.406171   
           False                       8                      0.280717   
                                       9                      0.510089   
2          True                        0                      0.280717   
           False                       1                      0.510089   
           True                        2                      0.510089   
                                       3                      0.148054   
                                       4                      0.510089   
                                       5                      0.280717   
                                       6                      0.510089   
                                       7                      0.510089   
                                       8                      0.510089   
                                       9                      0.148054   

category                                                                \
name                                              lambda_temperature_0   
Generation Is_From_Previous_Generation Individual                        
0          False                       0                      5.528275   
                                       1                     70.099327   
                                       2                     16.241361   
                                       3                      5.674832   
                                       4                     78.338013   
           True                        5                     48.748720   
                                       6                     95.423566   
                                       7                      0.771402   
                                       8                     43.243522   
                                       9                      7.045557   
1          True                        0                     43.243522   
           False                       1                      0.771402   
                                       2                      5.528275   
           True                        3                     43.243522   
           False                       4                     16.241361   
                                       5                     16.241361   
                                       6                     95.423566   
           True                        7                     48.748720   
   